In [1]:
import sys
import os
import os.path
import random
import collections
import shutil
import time
import glob
import csv
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image

ROOT_DIR = os.getcwd() + '/..'
%matplotlib inline

In [15]:
is_cuda = False

# paths
data_path = ROOT_DIR + '/data/sample' # '/data'
train_path = data_path + '/train/'
valid_path = data_path + '/val/'
# saved_model_path = ROOT_DIR + '/models/'
# submission_path = ROOT_DIR + '/submissions/'

# data
batch_size = 8
nb_train_samples = 20 # 25000
nb_valid_samples = 10 # 2000
# nb_test_samples  = 12500

# model
nb_runs = 1
nb_aug = 3
epochs = 2 #35
lr = 1e-4
clip = 0.001
pin_memory = True if is_cuda else False

In [3]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()
    end = time.time()
    
    # switch to train mode
    model.train()
    
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        target = target.cuda(async=True) if is_cuda else target
        image_var = torch.autograd.Variable(images)
        label_var = torch.autograd.Variable(target)

        # compute y_pred
        y_pred = model(image_var)
        loss = criterion(y_pred, label_var)

        # measure accuracy and record loss
        prec1, prec1 = accuracy(y_pred.data, target, topk=(1, 1))
        losses.update(loss.data[0], images.size(0))
        acc.update(prec1[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


In [4]:
def validate(val_loader, model, criterion, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (images, labels) in enumerate(val_loader):
        labels = labels.cuda(async=True) if is_cuda else labels
        image_var = torch.autograd.Variable(images, volatile=True)
        label_var = torch.autograd.Variable(labels, volatile=True)

        # compute y_pred
        y_pred = model(image_var)
        loss = criterion(y_pred, label_var)

        # measure accuracy and record loss
        prec1, temp_var = accuracy(y_pred.data, labels, topk=(1, 1))
        losses.update(loss.data[0], images.size(0))
        acc.update(prec1[0], images.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

    print('   * EPOCH {epoch} | Accuracy: {acc.avg:.3f} | Loss: {losses.avg:.3f}'.format(epoch=epoch,
                                                                                         acc=acc,
                                                                                         losses=losses))

    return acc.avg


In [5]:
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


In [6]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    global lr
    lr = lr * (0.1**(epoch // 30))
    for param_group in optimizer.state_dict()['param_groups']:
        param_group['lr'] = lr


def accuracy(y_pred, y_actual, topk=(1, )):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = y_actual.size(0)

    _, pred = y_pred.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(y_actual.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res


In [7]:
# model = models.vgg16(pretrained=True)
model = models.resnet152(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# model.classifier = nn.Sequential(
#     nn.Linear(25088, 4096),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(4096, 4096),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(4096, 2)
# )
model.fc = nn.Linear(2048, 2)

In [8]:
# model.features = torch.nn.DataParallel(model.features)
if is_cuda:
    model.cuda()
    cudnn.benchmark = True

In [9]:
traindir = train_path
valdir = valid_path
# testdir = test_path

In [10]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [11]:
train_loader = data.DataLoader(
    datasets.ImageFolder(traindir, transforms.Compose([
         transforms.RandomSizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         normalize])),
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=pin_memory)

val_loader = data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize])),
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=pin_memory)

criterion = nn.CrossEntropyLoss().cuda() if is_cuda else nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.classifier.parameters(), lr, weight_decay=1e-4)
optimizer = optim.Adam(model.fc.parameters(), lr, weight_decay=1e-4)

In [16]:
for epoch in range(epochs):
    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    prec1 = validate(val_loader, model, criterion, epoch)

   * EPOCH 0 | Accuracy: 100.000 | Loss: 0.375
   * EPOCH 1 | Accuracy: 100.000 | Loss: 0.373
